# Fast Agent - Build a SQL Agent fast!

<img src="./assets/LC_L1_top.png" align="left" width="500">

## Setup

In [0]:
%pip install -U mlflow>=3 langchain>=1 langchain.community tabulate databricks-langchain --upgrade --force-reinstall typing_extensions
dbutils.library.restartPython()

In [0]:
import mlflow

In [0]:
mlflow.langchain.autolog()

In [0]:
%sql
show catalogs

In [0]:
from langchain_core.tools import tool

@tool
def execute_sql(query: str) -> str:
    """Executa um comando SQL no Databricks e retorna os resultados."""
    try:
        result = spark.sql(query).limit(5).toPandas()
        return result.to_markdown()
    except Exception as e:
        return f"Error: {e}"

In [0]:
SYSTEM_PROMPT = """Você é um analista cuidadoso de Databricks SQL.
Regras:
- Pense passo a passo.
- Quando precisar de dados, chame a ferramenta `execute_sql` com UMA query SELECT.
- Apenas leitura; sem INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limite a 5 linhas de saída, a menos que o usuário peça mais.
- Se a ferramenta retornar 'Error:', revise o SQL e tente novamente.
- Prefira listas explícitas de colunas; evite SELECT *.
"""

In [0]:
from databricks_langchain import ChatDatabricks
from langchain.agents import create_agent

# Inicializar o modelo Databricks
llm = ChatDatabricks(
    endpoint="databricks-gpt-oss-120b",  # ou seu endpoint de serving
    temperature=0.1,
)

# Criar agente com o objeto do modelo
agent = create_agent(
    model=llm,
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
)

Here's a display of the agent ReAct Loop.

In [0]:
from IPython.display import Image, display

display(Image(agent.get_graph(xray=True).draw_mermaid_png()))

Run some queries. Notice:
- The agent does not have the database schema and will need to discover it independently.
- The agent may make mistakes! By returning error messages, the agent can self-correct its queries.
- Notice you invoke the agent with `agent.stream`.
    - This command and the `pretty_print` display the **messages** that communicate information between the model and the tools.
- Notice the agent doesn't remember the schema between invocations... More on this later!

In [0]:
question = "Qual tabela tem maior numero de registros no catalogo nasa_gcn e schema silver?"

for step in agent.stream(
    {"messages": question},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

In [0]:
question = "Liste todas as tabelas disponíveis no catálogo nasa_gcn e schema silver."

for step in agent.stream(
    {"messages": question},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

**Create your own query here!**  Add some questions of your own.

In [0]:
question = "Quais são os tipos de eventos mais frequentes no catálogo nasa_gcn e schema silver?"

for step in agent.stream(
    {"messages": question},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()